# Preliminary Analysis

### Data Cleaning Code
Code for cleaning and processing your data. Include a data dictionary for your transformed dataset.

- Data Dictionary for Air Quality
    - **indicator id:** id for each name
    - **name:** classify the sample in the air
    - **measure:** how the indicator is measured
    - **measure info:** information about the measure
    - **geo type name:** geography type, UHF stands for United Hospital Fund neighborhoods
    - **geo place name:** neighborhood name
    - **time period:** time frame
    - **start_date:** date started
    <br><br>
- Data Dictionary for Traffic Volume
    - **requestId:** unique id generated for each counts request
    - **boro:** lists which of the five diviions of New York City the location is within
    - **vol:** total sum of count collected within 15 minute increments
    - **segmentId:** The ID that idenifies each segment of a street
    - **wktgeom:** Geometry point of the location
    - **street:** street name of where traffic happened
    - **fromst:** start street of traffic
    - **tost:** end street where traffic volume was located
    - **direction:** text-based direction of traffic where the count took place
    - **date_time:** date at which it took place
    <br><br>
- Data Dictionary for 2020 mobility Dataset
    - **sub_region_2** which county it is
    - **date** date during recording
    - **retail_and_recreation_percent_change_from_baseline** mobility trends for places like restaurants, cafes, shopping centers, theme parks, museums, libraries, and movie theaters.
    - **grocery_and_pharmacy_percent_change_from_baseline** mobility trends for places like grocery markets, food warehouses, farmers markets, specialty food shops, drug stores, and pharmacies
    - **parks_percent_change_from_baseline** mobility trends for places like national parks, public beaches, marinas, dog parks, plazas, and public gardens
    - **transit_stations_percent_change_from_baseline** mobility trends for places like public transport hubs such as subway, bus, and train stations
    - **workplaces_percent_change_from_baseline** mobility trend for places of work
    - **residential_percent_change_from_baseline** mobility trends for places of residence
    
### Exploratory Analysis
Describe what work you have done so far and include the code. This may include descriptive statistics, graphs and charts, and preliminary models.

- We removed some columns that were irrelevant to what we want to predict as well as combine some columns that would fit together, such as the date and time.


### Challenges
Describe any challenges you've encountered so far. Let me know if there's anything you need help with!

- There were some challenges in figuring out what sort of data was necessary to include for our problem as it was targeted in New York City. 
- Figuring out the transformations to use on each dataset was also a challenge since there were many columns for each dataset and we had to find the ones that weren't relevant to our problem.
- There are some issues for the columns right now where there are some, such as segmentId in the Traffic Volume dataset where we are currently unsure if it's useful to keep or remove.
- Dealing with large datasets efficiently

### Future Work
Describe what work you are planning to complete for the final analysis.

- Future work includes using the cleaned data to use as inputs for models suited for classification such as Logisitc Regression and Linear Regression. 
- Make predictions using the models trained to obtain the accuracy scores to answer our questions
- Find the best model for accuracy as well as graph/chart the data to further understand it for future predictions.

### Contributions
Describe the contributions that each group member made.
- **Daniel Aguilar-Rodriguez**
    - Researched and acquired datasets
    - Helped present ideas during brainstorming session
    - Created jupyter notebook and helped clean datasets
    - Helped transform datasets and removed columns irrelevant to our work
    <br><br>
- **Jia Cong Lin**
    - Helped present ideas during brainstorming session
    - Helped define necessary columns for the mobility dataset
    - Assisted in determining columns to clean and define 
    <br><br>
- **Anvinh Truong**
    - Helped clean and define some columns for the datasets and dictionary
    - Helped present ideas during brainstorming session
    - Assisted in thinking of procedure to clean data columns

In [1]:
import pandas as pd
import numpy as np
import os
import requests
import datetime
import json
import re
import time 

In [2]:
def fetch_data(csv_name):
    url_dict = {'air_quality': 'https://data.cityofnewyork.us/api/views/c3uy-2p5r/rows.csv', 
                'mobility_global': 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv', 
                'traffic_volume': 'https://data.cityofnewyork.us/api/views/7ym2-wayt/rows.csv'}
    
    if csv_name == 'daily_air_quality':
        create_air_quality_csv()
    else:
        response = requests.get(url_dict[csv_name])
        path = f'datasets/{csv_name}.csv'
        with open(path, 'wb') as f:
            f.write(response.content)

In [3]:
def csv_exists(csv_name):
    path = f'datasets/{csv_name}.csv'
    file_exists = os.path.exists(path)
    return file_exists

In [4]:
def create_df(csv_name):
    if not csv_exists(csv_name):
        fetch_data(csv_name)
    path = f'datasets/{csv_name}.csv'
    df = pd.read_csv(path)
    return df

In [5]:
def mkdir_if_not_exist():
    directory = 'datasets'
    if not os.path.exists(f'{directory}/'):
        os.mkdir(directory)

In [6]:
def create_all_df(csv_names):
    mkdir_if_not_exist()
    df_list = []
    
    for csv_name in csv_names:
        print(f'Creating {csv_name} df')
        df = create_df(csv_name)
        df_list.append(df)
        
    return df_list

In [7]:
def get_json(url):
    response = requests.get(url)
    j = json.loads(response.text)
    
    return j

In [8]:
def get_code(target_value, url):
    code = ''
    j = get_json(url)
    
    for entry in j['Data']:
        if entry['value_represented'] == target_value:
            code = entry['code']
            
    return code

In [9]:
def get_state_code(state_name):
    url = 'https://aqs.epa.gov/data/api/list/states?email=test@aqs.api&key=test'
    state_code = get_code(state_name, url)

    return state_code

In [10]:
def get_county_code(state_code, county_name):
    url = f'https://aqs.epa.gov/data/api/list/countiesByState?email=test@aqs.api&key=test&state={state_code}'
    county_code = get_code(county_name, url)
    
    return county_code

In [11]:
'''def get_county_code_list(county_names):
    county_codes = []
    for county in county_names:
        county_codes.append()'''

def get_county_code_list(state_code):
    url = f'https://aqs.epa.gov/data/api/list/countiesByState?email=test@aqs.api&key=test&state={state_code}'

    j = get_json(url)
    county_codes = []
    
    for county in j['Data']:
        if re.fullmatch('Bronx|Kings|New York|Queens|Richmond', county['value_represented']):
            county_codes.append(county['code'])  
    
    return county_codes

In [12]:

#start_date = str(traffic_vol_daily['date'].min()).replace('-', '')
#end_date = str(traffic_vol_daily['date'].max()).replace('-', '')
#start_date = '20160101'
#end_date = '20160229'
#county_code = '005'

def get_daily_air_quality_list(state_code, county_codes):
    email = 'daguila000@citymail.cuny.edu'
    key = 'cobaltcrane81'
    param_code = '88101'
    daily_air_quality_list = []

    #year_counter = traffic_vol_daily['date'].min().year
    #end_year = traffic_vol_daily['date'].max().year
    year_counter = 2009
    end_year = 2020

    while year_counter <= end_year:
        for county_code in county_codes: 
            start_date = str(year_counter) + '0101'
            end_date = str(year_counter) + '1231'
            url = f'https://aqs.epa.gov/data/api/dailyData/byCounty?email={email}&key={key}&param={param_code}&bdate={start_date}&edate={end_date}&state={state_code}&county={county_code}'
            j = get_json(url)
            print(j['Header'])
            daily_air_quality_list.extend(j['Data'])
            time.sleep(6)
        year_counter += 1
        
    return daily_air_quality_list


In [13]:
'''state = 'New York'
state_code = get_state_code(state)
print(state_code)
print(get_county_code_list(state_code))'''

"state = 'New York'\nstate_code = get_state_code(state)\nprint(state_code)\nprint(get_county_code_list(state_code))"

In [16]:
def create_air_quality_csv():
    state = 'New York'
    state_code = get_state_code(state)
    county_code_list = get_county_code_list(state_code)
    daily_air_quality_list = get_daily_air_quality_list(state_code, county_code_list)
    daily_air_quality_df = pd.DataFrame(daily_air_quality_list)
    daily_air_quality_df.to_csv('datasets/daily_air_quality.csv')

In [ ]:
csv_names = ['traffic_volume', 'daily_air_quality']

traffic_volume, daily_air_quality = create_all_df(csv_names)

Creating traffic_volume df


## Air Quality Dataset Cleaning

print(air_quality.isnull().sum() / len(air_quality))

air_quality = air_quality.drop(['Message'], axis=1)
print(air_quality.isnull().sum() / len(air_quality))

print(air_quality.nunique() / len(air_quality))

air_quality = air_quality.drop(['Unique ID'], axis=1)
print(air_quality.shape)
print(air_quality.nunique() / len(air_quality))

air_quality = air_quality.drop(['Geo Join ID'], axis=1)
print(air_quality.shape)
print(air_quality.nunique() / len(air_quality))

air_quality.dtypes

air_quality.nunique()

air_quality['Time Period'].unique()

np.sort(air_quality['Time Period'].unique())

air_quality['Time Period'].value_counts() / len(air_quality)

air_quality['Start_Date'].unique()

air_quality['Start_Date'] = pd.to_datetime(air_quality['Start_Date'], infer_datetime_format=True)

air_quality['Start_Date'].min()

air_quality['Start_Date'].value_counts().sort_index() / len(air_quality)

air_quality.groupby('Start_Date')['Time Period'].value_counts()

list(air_quality[air_quality['Time Period'].str.contains('Winter')]['Time Period'].unique())

air_quality[air_quality['Time Period'].str.contains('Winter|Summer')].groupby('Time Period')['Start_Date'].value_counts()

air_quality[air_quality['Time Period'].str.contains('Winter')]

air_quality.sample(10)

air_quality[air_quality['Geo Type Name'].str.contains('Borough')]['Time Period'].unique()

air_quality[air_quality['Geo Type Name'].str.contains('Borough')].groupby('Name')['Geo Place Name'].value_counts()

list(air_quality['Name'].unique())

air_quality.groupby(['Name', 'Start_Date'])['Time Period'].value_counts()

air_quality['Geo Type Name'].unique()

air_quality[air_quality['Geo Type Name'].str.contains('CD')]

air_quality[air_quality['Geo Type Name'].str.contains('UHF42')]

air_quality[air_quality['Geo Type Name'].str.contains('UHF34')]

air_quality_boros = air_quality[air_quality['Geo Type Name'].str.contains('Borough')]
air_quality_boros = air_quality_boros.drop(['Geo Type Name'], axis=1)

air_quality_boros.sample(10)

for time_period in air_quality_boros['Time Period'].sort_values().unique():
    print(time_period, 'length:', len(time_period))

air_quality.groupby('Time Period')['Start_Date'].value_counts()

def create_end_dates(air_quality_boros_dict):
    end_dates = []
    for row in air_quality_boros_dict:
        time_period_str_len = len(row['Time Period'])
        year = row['Start_Date'].year
        if time_period_str_len == 4:
            date = datetime.date(year, 12, 31)
        elif time_period_str_len == 9:
            date = datetime.date(year + 2, 12, 31)
        elif time_period_str_len == 11:
            date = datetime.date(year, 8, 31)
        elif time_period_str_len == 14:
            date = datetime.date(year + 1, 2, 28)
        elif time_period_str_len == 19:
            year = int(row['Time Period'][-4:])
            date = datetime.date(year, 12, 31)
        end_dates.append(date)
    return end_dates

air_quality_boros['end_date'] = create_end_dates(air_quality_boros.to_dict('records'))

air_quality_boros.sample(10)

air_quality_boros['Indicator ID'].nunique()

print(air_quality.groupby(['Name', 'Measure'])['Indicator ID'].value_counts())
print(len(air_quality.groupby(['Name', 'Measure'])['Indicator ID'].value_counts()))

air_quality_boros = air_quality_boros.drop(['Indicator ID'], axis=1)

air_quality_boros.sample(10)

air_quality_boros.groupby(['Name'])['Measure'].value_counts()

air_quality_boros[air_quality_boros['Time Period'].str.contains("Annual")]['Measure'].unique()

air_quality_boros[air_quality_boros['Time Period'].str.contains("Annual")]['Name'].unique()

air_quality_boros[air_quality_boros['Measure'].str.contains("Mean")]['Time Period'].unique()

## Traffic Volume Dataset Cleaning

In [ ]:
traffic_volume.sample(10)

In [ ]:
traffic_volume.shape

In [ ]:
print(traffic_volume.isnull().sum() / len(traffic_volume))

In [ ]:
print(traffic_volume.nunique() / len(traffic_volume))

In [ ]:
traffic_volume.nunique()

In [ ]:
traffic_volume.dtypes

In [ ]:
traffic_volume.Yr.min()

In [ ]:
traffic_volume = traffic_volume[traffic_volume['Yr'] >= 2005]

In [ ]:
traffic_volume.shape

In [ ]:
traffic_volume['Yr'].value_counts().sort_index()

In [ ]:
traffic_volume = traffic_volume[traffic_volume['Yr'] > 2008]

In [ ]:
traffic_volume.shape

In [ ]:
traffic_volume['date_time'] = pd.to_datetime(dict(year=traffic_volume.Yr, \
                                                  month=traffic_volume.M, \
                                                  day=traffic_volume.D, \
                                                  hour=traffic_volume.HH, \
                                                  minute=traffic_volume.MM))

In [ ]:
traffic_volume = traffic_volume.drop(['Yr', 'M', 'D', 'HH', 'MM'], axis=1)

In [ ]:
traffic_volume.sample(10)

In [ ]:
traffic_volume['date_time'].dt.year.value_counts().sort_index()

In [ ]:
traffic_volume['date_time'].dt.year.value_counts().sort_index() / len(traffic_volume)

In [ ]:
traffic_volume.head()

In [ ]:
traffic_volume.sort_values(["date_time"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True);
traffic_volume.head(15)

In [ ]:
traffic_vol_daily = traffic_volume.groupby(['Boro', traffic_volume['date_time'].dt.date])['Vol'].mean().reset_index()
traffic_vol_daily.columns = traffic_vol_daily.columns.str.lower()
traffic_vol_daily.rename(columns={'date_time':'date'}, inplace=True)

In [ ]:
traffic_vol_daily.info()

In [ ]:
print(len(traffic_vol_daily[traffic_vol_daily['boro'] == 'Bronx']))
traffic_vol_daily[traffic_vol_daily['boro'] == 'Bronx'].head(10)

In [ ]:
print(len(traffic_vol_daily[traffic_vol_daily['boro'] == 'Queens']))
traffic_vol_daily[traffic_vol_daily['boro'] == 'Queens'].head(10)

In [ ]:
print(traffic_vol_daily[traffic_vol_daily['boro'] == 'Brooklyn'].shape[0])
traffic_vol_daily[traffic_vol_daily['boro'] == 'Brooklyn'].head(10)

In [ ]:
print(traffic_vol_daily[traffic_vol_daily['boro'] == 'Manhattan'].shape[0])
traffic_vol_daily[traffic_vol_daily['boro'] == 'Manhattan'].head(10)

In [ ]:
print(traffic_vol_daily[traffic_vol_daily['boro'] == 'Staten Island'].shape[0])
traffic_vol_daily[traffic_vol_daily['boro'] == 'Staten Island'].head(10)

## EPA data

In [ ]:
#daily_air_quality_df = pd.DataFrame(daily_air_quality_list)

In [ ]:
#daily_air_quality_df.to_csv('datasets/daily_air_quality.csv')

In [ ]:
daily_air_quality_df = daily_air_quality

In [ ]:
print(daily_air_quality_df.info())

In [ ]:
daily_air_quality_df.sample(5).transpose()

In [ ]:
daily_air_quality_df['date_local'] = pd.to_datetime(daily_air_quality_df['date_local'], infer_datetime_format=True)
daily_air_quality_df['date_local'] = daily_air_quality_df['date_local'].dt.date

In [ ]:
for county in daily_air_quality_df['county'].unique():
    mean = round(daily_air_quality_df[daily_air_quality_df['county'] == county]['aqi'].mean())
    filled_na_county = daily_air_quality_df.loc[(daily_air_quality_df['county'] == county) & (daily_air_quality_df['aqi'].isnull()), ['aqi']].fillna(mean)
    daily_air_quality_df.loc[(daily_air_quality_df['county'] == county) & (daily_air_quality_df['aqi'].isnull()), ['aqi']] = filled_na_county

In [ ]:
for county in daily_air_quality_df['county'].unique():
    mean = daily_air_quality_df[(daily_air_quality_df['county'] == county) & 
                                (daily_air_quality_df['sample_duration'] != '1 HOUR')]['arithmetic_mean'].mean()
    
    daily_air_quality_df.loc[(daily_air_quality_df['county'] == county) & 
                             (daily_air_quality_df['sample_duration'] == '1 HOUR'), ['arithmetic_mean']] = mean

In [ ]:
'''import math 
for aqi in daily_air_quality_df['aqi']:
    if not aqi.is_integer() and not math.isnan(aqi):
        print('is not integer', aqi)
        break'''

In [ ]:
daily_air_quality_df['aqi'] = daily_air_quality_df['aqi'].astype(int)

In [ ]:
daily_air_quality_df = daily_air_quality_df[(daily_air_quality_df['date_local'] >= traffic_vol_daily['date'].min()) & 
                                            (daily_air_quality_df['date_local'] <= traffic_vol_daily['date'].max())]

In [ ]:
daily_air_quality_df.info()

In [ ]:
print(len(daily_air_quality_df))
print(len(traffic_vol_daily))

In [ ]:
daily_air_quality_df['validity_indicator'].value_counts()

In [ ]:
daily_air_quality_df = daily_air_quality_df[daily_air_quality_df['validity_indicator'] == 'Y']

In [ ]:
daily_air_quality_df['sample_duration'].value_counts()

In [ ]:
daily_air_quality_df['sample_duration'].value_counts() / len(daily_air_quality_df['sample_duration'])

In [ ]:
daily_air_quality_df['event_type'].unique()

In [ ]:
daily_air_quality_df['event_type'].value_counts()

In [ ]:
daily_air_quality_df['parameter'].unique()

In [ ]:
daily_air_quality_df['method'].unique()

In [ ]:
daily_air_quality_df['units_of_measure'].unique()

In [ ]:
daily_air_quality_df_cleaned = daily_air_quality_df[['date_local', 
                                                     'parameter',
                                                     'units_of_measure',  
                                                     'arithmetic_mean', 
                                                     'first_max_value', 
                                                     'aqi', 
                                                     'county']]

In [ ]:
daily_air_quality_df_cleaned = daily_air_quality_df_cleaned.reset_index().drop(['index'], axis=1)

In [ ]:
daily_air_quality_df_cleaned.sample(15)

In [ ]:
daily_air_quality_df_cleaned = daily_air_quality_df_cleaned.groupby(['county', 'date_local'])['arithmetic_mean', 'aqi'].mean().reset_index()
daily_air_quality_df_cleaned

In [ ]:
daily_air_quality_df_cleaned.rename(columns={'county': 'boro', 'date_local': 'date'}, inplace=True)

In [ ]:
daily_air_quality_df_cleaned.info()

In [ ]:
county_boro = {'New York': 'Manhattan', 'Kings': 'Brooklyn', 'Richmond': 'Staten Island'}

for county in county_boro:    
    daily_air_quality_df_cleaned.loc[(daily_air_quality_df_cleaned['boro'] == county), ['boro']] = county_boro[county]

In [ ]:
daily_air_quality_df_cleaned.boro.unique()

In [ ]:
daily_air_quality_df_cleaned = daily_air_quality_df_cleaned.astype({'boro': 'category'})

In [ ]:
daily_air_quality_df_cleaned.dtypes

## Combining Traffic Volume and Daily AQI

In [ ]:
traffic_vol_aqi = pd.merge(traffic_vol_daily, daily_air_quality_df_cleaned, how='outer', on=['boro','date'])

In [ ]:
traffic_vol_aqi.info()

In [ ]:
traffic_vol_aqi.isnull().sum() / len(traffic_vol_aqi)

In [ ]:
for measure in traffic_vol_aqi.columns[-3:]:
    print(measure)

In [ ]:
for measure in traffic_vol_aqi.columns[-3:]:
    for boro in traffic_vol_aqi['boro'].unique():
        mean = traffic_vol_aqi[traffic_vol_aqi['boro'] == boro][measure].mean()
        
        if measure == 'aqi':
            mean = round(mean)
            
        filled_na_boro = traffic_vol_aqi.loc[(traffic_vol_aqi['boro'] == boro) & 
                                             (traffic_vol_aqi[measure].isnull()), [measure]].fillna(mean)
        
        traffic_vol_aqi.loc[(traffic_vol_aqi['boro'] == boro) & (traffic_vol_aqi[measure].isnull()), [measure]] = filled_na_boro

In [ ]:
traffic_vol_aqi.isnull().sum()

## Transformed Datasets

In [ ]:
daily_air_quality_df_cleaned.sample(10)

In [ ]:
traffic_vol_daily.sample(10)

In [ ]:
traffic_vol_aqi.sample(10)